# oneTBB flow graph

##### Sections
- [oneTBB Flow Graph](#oneTBB-Flow-Graph)
- _Code_: [Forward substitution using a oneTBB flow graph](#Forward-substitution-using-a-oneTBB-flow-graph)

## Learning Objectives
* Understand the basic steps of constructing and executing a oneTBB flow graph

# oneTBB Flow Graph

The oneTBB library provides functions and classes for applications that can be expressed as graphs
of computations. These classes and functions are in namespace ``tbb::flow``. A flow graph is used when you want to be precise about the dependencies in your code, or if you have a streaming
application that requires more than just the simple linear pipeline supported by ``tbb::parallel_pipeline``.
You can read more about the oneTBB flow graph interfaces in the 
[flow graph](https://spec.oneapi.com/versions/latest/elements/oneTBB/source/flow_graph.html) section of 
the oneAPI specification.  

## Forward substitution using a oneTBB flow graph

Here we implement the forward substitution method of solving a set of 
equations ``Ax = b``, where ``A`` is an NxN lower triangular matrix. To do so, we create a
simple flow graph example that uses only a single node type, ``continue_node``. 

As shown below in the diagram of the dependencies in forward substitution, once the value of 
element a<sub>i</sub><sub>,j</sub> is known, both a<sub>i+1</sub><sub>,j</sub> and a<sub>i</sub><sub>,j+1</sub>
can be solved for.  These dependencies are shown in figure (a) below. To reduce scheduling overheads in
parallel implementations, we can aggregate elements into blocks to schedule as a task, where once the 
elements in block i,j are available, then all the elements in blocks i+1,j and i,j+1 can be solved for.

![A tiled dependency graph implementation of forward substitution](img/fwd_sub.png)

### Run the sequential baseline implementation

Inspect the sequential implementation of forward substitution below - there are no modifications necessary. This code implements 
a serial blocked implementation of forward substitution. Run the first cell to create the file, then run the cell below it to compile 
and execute the code. This represents the baseline for the computation that we will convert into a oneTBB flow graph.

1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run the baseline__ section below the code snippet to compile and execute the code in the saved file
3. Inspect the two new files ``serial_pipeline_before.txt`` and ``serial_pipeline_after.txt`` to see the results of changing the case of the text.

In [ ]:
%%writefile lab/fwd-sub-serial.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <iostream>
#include <thread>
#include <vector>


// defined in common/fwd_sub.cpp
std::vector<double> init_fwd_sub(std::vector<double>& x,
                                 std::vector<double>& a,
                                 std::vector<double>& b); 

void check_fwd_sub(const std::vector<double>& x,
                   const std::vector<double>& x_gold); 
        
void fwd_sub_serial(std::vector<double>& x, const std::vector<double>& a, std::vector<double>& b) {
  const int N = x.size();
  const int block_size = 512;
  const int num_blocks = N / block_size;

  for ( int r = 0; r < num_blocks; ++r ) {
    for ( int c = 0; c <= r; ++c ) {
      int i_start = r*block_size, i_end = i_start + block_size;
      int j_start = c*block_size, j_max = j_start + block_size - 1;
      for (int i = i_start; i < i_end; ++i) {
        int j_end = (i <= j_max) ? i : j_max + 1;
        for (int j = j_start; j < j_end; ++j) {
          b[i] -= a[j + i*N] * x[j];
        }
        if (j_end == i) {
          x[i] = b[i] / a[i + i*N];
        }
      }
    }
  }
}

int main() {
  const int N = 32768;

  std::vector<double> a(N*N);
  std::vector<double> b(N);
  std::vector<double> x(N);

  auto x_gold = init_fwd_sub(x,a,b);

  double serial_time = 0.0;
  {
    auto st0 = std::chrono::high_resolution_clock::now();
    fwd_sub_serial(x,a,b);
    serial_time = 1e-9*(std::chrono::high_resolution_clock::now() - st0).count();
  }
  check_fwd_sub(x, x_gold);
  std::cout << "serial_time == " << serial_time << " seconds" << std::endl;
  return 0;
}


### Build and Run the baseline code

Select the cell below and click Run ▶ to compile and execute the code that you modified above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_fwd-sub-serial.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_fwd-sub-serial.sh; else ./scripts/run_fwd-sub-serial.sh; fi

### Implement a parallel version with tbb::flow::graph

In this section, you will use a ``tbb::flow::graph`` to create a parallel implementation of forward substitution. The classes and functions
for creating oneTBB flow graphs are in namespace ``tbb::flow``.

There are five general steps for creating and running a graph.

1. Create an instance of class ``tbb::flow::graph``. In our example, we use an object named ``g``.
2. Create the node objects. In this example, we will only use ``tbb::flow::continue_node`` objects. One object will be created per block of elements.
3. Make edges between the nodes from predecessor to successor using ``tbb::flow::make_edge(predecessor_node, successor_node)``. In this example, each node will be connected to nodes i+1,j and i,j+1. This enforces the ordering necessary for a legal execution of forward substitution.
4. Start the activity in the graph. In this example, we will invoke try_put on the top corner of the graph, i.e. "nodes[0]->try_put(tbb::flow::continue_msg{})``.
5. Wait for the graph to complete, by calling ``g.wait_for_all()``.

You can find more information about tbb::flow::continue_node [here](https://spec.oneapi.com/versions/latest/elements/oneTBB/source/flow_graph/continue_node_cls.html). The key points for our example
are:

The constructor arguments:

```cpp
template<typename Body>
continue_node( graph &g, Body body );
```

And the [requirements on the body](https://spec.oneapi.com/versions/latest/elements/oneTBB/source/named_requirements/flow_graph/continue_node_body.html):

```cpp
continue_msg Body::operator()(const continue_msg &v) const
```

A `continue_msg` is an empty message, differentiating it from other value-carrying messages supported by the flow graph 
interfaces. 

For this exercise, complete the following steps:

1. Inspect the code cell below and make the following modifications.
  1. Add the body to the ``continue_node`` created in function ``create_node``. The function ``create_node`` is called repeatedly in our example.
  2. Add a call to create_edges in the loop in function ``fwd_sub_parallel``.
  3. Invoke ``nodes[0]->try_put(tbb::flow::continue_msg{})`` to start the graph.
  4. Add a call to ``g.wait_for_all()`` after the graph has been started.
2. When the modifications are complete, click run ▶ to save the code to a file.
3. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file.

In [ ]:
%%writefile lab/fwd-sub-parallel.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <iostream>
#include <thread>
#include <vector>

#include <tbb/tbb.h>


// defined in common/fwd_sub.cpp
std::vector<double> init_fwd_sub(std::vector<double>& x,
                                 std::vector<double>& a,
                                 std::vector<double>& b); 

void check_fwd_sub(const std::vector<double>& x,
                   const std::vector<double>& x_gold); 

using Node = tbb::flow::continue_node<tbb::flow::continue_msg>;
using NodePtr = std::shared_ptr<Node>;

NodePtr create_node(tbb::flow::graph& g, int r, int c, int block_size, 
                   std::vector<double>& x, const std::vector<double>& a, 
                   std::vector<double>& b) {

  const int N = x.size();
  return std::make_shared<Node>(g,
    [r, c, block_size, N, &x, &a, &b] (const tbb::flow::continue_msg& msg) {
      // STEP A: Add node body
      return msg;
    }
  );
}

void add_edges(std::vector<NodePtr>& nodes, int r, int c, int block_size, int num_blocks) {
  NodePtr np = nodes[r*num_blocks + c];
  if (c + 1 < num_blocks && r != c)
    tbb::flow::make_edge(*np, *nodes[r*num_blocks + c + 1]);
  if (r + 1 < num_blocks)
    tbb::flow::make_edge(*np, *nodes[(r + 1)*num_blocks + c]);
}

void fwd_sub_parallel(std::vector<double>& x, const std::vector<double>& a, 
              std::vector<double>& b) {
  const int N = x.size();
  const int block_size = 1024;
  const int num_blocks = N / block_size;

  std::vector<NodePtr> nodes(num_blocks*num_blocks);
  tbb::flow::graph g;
  for (int r = num_blocks - 1; r >= 0; --r) {
    for (int c = r; c >= 0; --c) {
      nodes[r*num_blocks + c] = create_node(g, r, c, block_size, x, a, b);;
      // STEP B: Add a call to function add_edges
    }
  }
  // STEP C: send a tbb::flow::continue_msg{} to nodes[0]
  // STEP D: wait for the graph to complete
}

int main() {
  const int N = 32768;

  std::vector<double> a(N*N);
  std::vector<double> b(N);
  std::vector<double> x(N);

  auto x_gold = init_fwd_sub(x,a,b);

  double parallel_time = 0.0;
  {
    auto pt0 = std::chrono::high_resolution_clock::now();
    fwd_sub_parallel(x,a,b);
    parallel_time = 1e-9*(std::chrono::high_resolution_clock::now() - pt0).count();
  }
  check_fwd_sub(x, x_gold);
  std::cout << "parallel_time == " << parallel_time << " seconds" << std::endl;
  return 0;
}

### Build and Run the modified code

Select the cell below and click Run ▶ to compile and execute the code that you modified above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_fwd-sub-parallel.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_fwd-sub-parallel.sh; else ./scripts/run_fwd-sub-parallel.sh; fi

## Forward Substitution Solution (Don't peak unless you have to)

In [ ]:
%%writefile solutions/fwd-sub-solution.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <iostream>
#include <thread>
#include <vector>

#include <tbb/tbb.h>


// defined in common/fwd_sub.cpp
std::vector<double> init_fwd_sub(std::vector<double>& x,
                                 std::vector<double>& a,
                                 std::vector<double>& b); 

void check_fwd_sub(const std::vector<double>& x,
                   const std::vector<double>& x_gold); 

using Node = tbb::flow::continue_node<tbb::flow::continue_msg>;
using NodePtr = std::shared_ptr<Node>;

NodePtr create_node(tbb::flow::graph& g, int r, int c, int block_size, 
                   std::vector<double>& x, const std::vector<double>& a, 
                   std::vector<double>& b) {

  const int N = x.size();
  return std::make_shared<Node>(g,
    [r, c, block_size, N, &x, &a, &b] (const tbb::flow::continue_msg& msg) {
      int i_start = r*block_size, i_end = i_start + block_size;
      int j_start = c*block_size, j_max = j_start + block_size - 1;
      for (int i = i_start; i < i_end; ++i) {
        int j_end = (i <= j_max) ? i : j_max + 1;
        for (int j = j_start; j < j_end; ++j) {
          b[i] -= a[j + i*N] * x[j];
        }
        if (j_end == i) {
          x[i] = b[i] / a[i + i*N];
        }
      }
      return msg;
    }
  );
}

void add_edges(std::vector<NodePtr>& nodes, int r, int c, int block_size, int num_blocks) {
  NodePtr np = nodes[r*num_blocks + c];
  if (c + 1 < num_blocks && r != c)
    tbb::flow::make_edge(*np, *nodes[r*num_blocks + c + 1]);
  if (r + 1 < num_blocks)
    tbb::flow::make_edge(*np, *nodes[(r + 1)*num_blocks + c]);
}

void fwd_sub_parallel(std::vector<double>& x, const std::vector<double>& a, 
              std::vector<double>& b) {
  const int N = x.size();
  const int block_size = 1024;
  const int num_blocks = N / block_size;

  std::vector<NodePtr> nodes(num_blocks*num_blocks);
  tbb::flow::graph g;
  for (int r = num_blocks - 1; r >= 0; --r) {
    for (int c = r; c >= 0; --c) {
      nodes[r*num_blocks + c] = create_node(g, r, c, block_size, x, a, b);;
      add_edges(nodes, r, c, block_size, num_blocks);
    }
  }
  nodes[0]->try_put(tbb::flow::continue_msg{});
  g.wait_for_all();
}

int main() {
  const int N = 32768;

  std::vector<double> a(N*N);
  std::vector<double> b(N);
  std::vector<double> x(N);

  auto x_gold = init_fwd_sub(x,a,b);

  double parallel_time = 0.0;
  {
    auto pt0 = std::chrono::high_resolution_clock::now();
    fwd_sub_parallel(x,a,b);
    parallel_time = 1e-9*(std::chrono::high_resolution_clock::now() - pt0).count();
  }
  check_fwd_sub(x, x_gold);
  std::cout << "parallel_time == " << parallel_time << " seconds" << std::endl;
  return 0;
}

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_fwd-sub-solution.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_fwd-sub-solution.sh; else ./scripts/run_fwd-sub-solution.sh; fi

## Next steps

If you are ready, go to [the next module](../04_oneTBB_concurrent_containers/oneTBB_concurrent_containers.ipynb).